In [1]:
# Initial Setup.
import pandas as pd
import re
import numpy as np
import math
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
drive.mount('/content/gdrive')
path = '/content/gdrive/My Drive/Mastodon_SaiGinny/'

Mounted at /content/gdrive


In [2]:
#change the input

data=pd.read_csv(path+'data_collection/0_355_normalized.csv',converters={'data.most_recent_tweet_id': str})

In [3]:
data.head()

,data.verified,data.location,data.receives_your_dm,data.protected,data.created_at,data.verified_type,data.name,data.subscription_type,data.pinned_tweet_id,data.profile_image_url,...,data.public_metrics.followers_count,data.public_metrics.following_count,data.public_metrics.tweet_count,data.public_metrics.listed_count,data.public_metrics.like_count,data.entities.description.mentions,data.url,data.entities.url.urls,data.entities.description.hashtags,data.entities.description.urls
0,False,Berlin,False,False,2009-04-08T14:53:42.000Z,none,dmnk@infosec.exchange,NaN,1.388135e+18,https://pbs.twimg.com/profile_images/137940817...,...,4342,512,13399,69,25640,"[{'start': 51, 'end': 63, 'username': 'aflplus...",NaN,NaN,NaN,NaN
1,False,"Strasbourg, FR",False,False,2017-07-13T08:59:52.000Z,none,Claudia,NaN,1.831641e+18,https://pbs.twimg.com/profile_images/125161648...,...,2749,725,778,15,6693,"[{'start': 13, 'end': 24, 'username': 'CamBioc...",https://t.co/gsXVIt526i,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...","[{'start': 26, 'end': 37, 'tag': 'ERCGrantee'}...",NaN
2,False,NaN,False,False,2014-02-09T17:45:54.000Z,none,Tero-Petri Ruoko,NaN,NaN,https://pbs.twimg.com/profile_images/719790981...,...,282,512,253,2,1405,NaN,NaN,NaN,NaN,NaN
3,True,"Twin Cities, Minnesota",False,False,2020-01-16T14:12:24.000Z,blue,Alan Rozenshtein,NaN,NaN,https://pbs.twimg.com/profile_images/121783634...,...,3935,628,5728,81,6816,"[{'start': 11, 'end': 26, 'username': 'UofMNLa...",https://t.co/GaCoPeTLKG,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...",NaN,NaN
4,False,Gadfly in the Plutocracy,False,False,2021-03-23T18:58:46.000Z,none,Ashley M. Gjøvik,NaN,1.805006e+18,https://pbs.twimg.com/profile_images/164398319...,...,16315,3206,12004,95,29858,NaN,https://t.co/PURtXIXMpd,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...",NaN,NaN


In [11]:
import requests
import os
import json
import pandas as pd
import time

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAADi9vgEAAAAALlGqsqiDThx1zhq1VNBmj1LY8c0%3DfxaDd86f5o1v6iBYeMCU9l0OwJPXm1KcIqEYqO2GgUYqAMTyto'


def create_url(id):
    # Specify the usernames that you want to lookup below
    # You can enter up to 100 comma-separated values.

    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    url = "https://api.twitter.com/2/tweets?ids={}&tweet.fields=created_at".format(id)
    #url = "https://api.twitter.com/1.1/users/search.json?q={}&{}".format(usernames, user_fields)
    return url


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2TweetLookupPython"
    return r


def connect_to_endpoint(url):
    while True:
        response = requests.request("GET", url, auth=bearer_oauth)
        time.sleep(10)
        print(response.status_code)
        if response.status_code == 429:
            print("429 Too Many Requests - Waiting for 15 minutes before retrying...")
            time.sleep(900)  # 1 minute
        elif response.status_code != 200:
            raise Exception(
                "Request returned an error: {} {}".format(
                    response.status_code, response.text
                )
            )
        else:
            return response.json()


ids = data['data.most_recent_tweet_id'][15:29].tolist()
all_data = []

for id in ids:
    url = create_url(id)
    json_response = connect_to_endpoint(url)
    all_data.append(json_response)

df = pd.DataFrame(all_data)
df



429
429 Too Many Requests - Waiting for 15 minutes before retrying...


KeyboardInterrupt: 

In [10]:
import requests
import time

response = requests.request("GET", "https://api.twitter.com/2/tweets/1826903925649199104?tweet.fields=created_at", auth=bearer_oauth)

rate_limit = response.headers.get('x-rate-limit-limit')
rate_remaining = response.headers.get('x-rate-limit-remaining')
rate_reset = response.headers.get('x-rate-limit-reset')
if rate_reset is not None:
    rate_reset_time = time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(int(rate_reset)))
else:
    rate_reset_time = 'N/A'

print(f"Rate Limit: {rate_limit}")
print(f"Remaining Requests: {rate_remaining}")
print(f"Rate Limit Resets At (UTC): {rate_reset_time}")


Rate Limit: 15
Remaining Requests: 14
Rate Limit Resets At (UTC): 2024-10-10 15:27:07


In [7]:
df = pd.DataFrame(all_data)

In [14]:
df

,data
0,"[{'created_at': '2024-10-05T14:32:28.000Z', 'i..."
1,[{'text': 'I loved my time at @Leopoldina! Ama...
2,"[{'id': '1826903925649199160', 'created_at': '..."


In [9]:
for i in df['data']:
  print(i)

[{'text': '@TheRocketMan_1 @raistolo @ErrataRob https://t.co/klmjRqVTDa', 'edit_history_tweet_ids': ['1842573576941261272'], 'id': '1842573576941261272', 'created_at': '2024-10-05T14:32:28.000Z'}]
[{'text': 'I loved my time at @Leopoldina! Amazing talks and interactions! Thanks @CarellThomas for the invitation 🤩 https://t.co/g3eFrIq5Jc', 'edit_history_tweet_ids': ['1839885698523902445'], 'id': '1839885698523902445', 'created_at': '2024-09-28T04:31:48.000Z'}]
[{'id': '1826903925649199160', 'text': 'Optical spectroscopy: https://t.co/U5zFlrFpSu', 'edit_history_tweet_ids': ['1826903925649199160'], 'created_at': '2024-08-23T08:46:52.000Z'}]
[{'created_at': '2024-10-06T01:23:54.000Z', 'edit_history_tweet_ids': ['1842737513389015436'], 'text': 'RT @LuisParrales_: Unlike most of Amy Wax’s defenders, @JohnHMcWhorter rightly gets that it’s possible to both a) uphold free speech and b)…', 'id': '1842737513389015436'}]
[{'id': '1842599813688504320', 'created_at': '2024-10-05T16:16:44.000Z', 'text

In [8]:
df.to_csv(path+'recent_tweet0_15.csv', index=False)